# Load the Phosphorus System

In [ ]:
# Uncomment when running in Google Colab
#!pip install git+https://github.com/EasyArray/ling516.git@v4

In [ ]:
%config InteractiveShell.ast_node_interactivity='all'
from phosphorus import *

# PhiValues


In [ ]:
PhiValue('LAUGHS(A)')
PhiValue('LAUGHS(A)', stype=Type.t) #explicit type
PhiValue('LAUGHS(A).t')             #annotated type
PhiValue('lambda x=e: LAUGHS(x).t') #annotate lambda parameters using defaults

## Backtick DSL

In [ ]:
`LAUGHS(A)
`LAUGHS(A).t
`LAUGHS(A) and CRIES(A)             #inferred type of BoolOp
`lambda x=e: LAUGHS(x).t
`lambda x=e: LAUGHS(x) and CRIES(x) #inferred type projects

## Beta Reduction

In [ ]:
`(lambda x=e: LAUGHS(x).t)('A'.e) #correct type inference
`(lambda x=t: LAUGHS(x).t)('A'.e) #type mismatch warning

## Prepopulated domain: capital letters

In [ ]:
A,B,C,D,*_ = DOMAIN
A
B
C
D

# Simplification Beyond Beta Reduction

In [ ]:
# Inlining of name values
f = `(lambda x: LAUGHS(x).t)
`f(A)

In [ ]:
# Other simplifications
`True and foo
`foo and True
`(g|{x:foo})[x]
`{x:z} | {x:foo}

## Guards / Restrictions / Domain Restrictions

In [ ]:
`foo % True
`foo % False

deciduous = `lambda x=e: DECIDUOUS(x) % TREE(x)
deciduous(A)          # Correct beta reduction

`foo(bar % baz) % baz # Elimation of redundant guards

# Trees, including semantic annotations

In [ ]:
t1 = Tree.fromstring('(S (NP John) (VP (V laughs)))')
t1
t1[0][0].sem = `John.e
t1[1][0].sem = `lambda x=e: LAUGHS(x).t
t1


# Semantic Interpretation

In [ ]:
lexicon = {
  "john"  : `JOHN.e,
  "mary"  : `MARY.e,
  "kaline" : `KALINE.e,
  "cat"   : `lambda x=e: CAT(x).t,
  "gray"  : `lambda x=e: GRAY(x).t,
  "runs"  : `lambda x=e: RUN(x).t,
  "loves" : `lambda y=e: (lambda x=e: LOVE(x,y).t),
  "the"   : `lambda f=et: iota(f).e % singular(f),
}

calc = Interpreter(lexicon=lexicon)

In [ ]:
# Trace and Pronouns (TP)
@calc.rule()
def TP(*, alpha: str):
  try:
    i = int(alpha.split('_')[1])  # is it indexed?
    return `g[i].e
  except: pass
  
  return UNDEF

# Predicate Abstraction (PA)
@calc.rule()
def PA(beta: PhiValue, *, alpha: str):
  try:
    i = int(alpha[0])       # is it an index?
    if beta.stype == Type.t:
      return `lambda x=e: beta(g=g|{i:x}).t
  except: pass
  
  return UNDEF

# Terminal Node (TN): lexical lookup of alpha itself
@calc.rule()
def TN(*, alpha: str):
  return calc.lookup(alpha) # returns VACUOUS if not found

# Non-Branching Node (NN): pass child meaning unchanged
@calc.rule()
def NN(beta: PhiValue):
  return beta

# Functional Application (FA): apply function to argument
@calc.rule()
def FA(beta: PhiValue, gamma: PhiValue):
  if UNDEF in (beta, gamma):
    return UNDEF
  
  # Determine order
  if takes(beta, gamma):
    fn, arg = beta, gamma
  elif takes(gamma, beta):
    fn, arg = gamma, beta
  else:
    return UNDEF

  # Note that if arg fails the fn guard, it will already return UNDEF:
  return `fn(arg) % defined(arg) 

# Predicate Modification (PM): conjoin meanings
@calc.rule()
def PM(beta: PhiValue, gamma: PhiValue):
  if UNDEF in (beta, gamma):
    return UNDEF
  
  if beta.stype == gamma.stype == Type.et:
    return `(lambda x=e: beta(x) and gamma(x) 
              % defined(beta(x)) % defined(gamma(x))
    )

  return UNDEF

## Predicate Modification

In [ ]:
t0 = Tree.fromstring('(S Kaline (VP is (NP (D a) (NP gray (N cat)))))')
t0
calc.interpret(t0)
t0

## Domain Restriction Projection

In [ ]:
t2 = Tree.fromstring('(S (NP (D the) (N cat)) (VP (V runs)))')
calc.interpret(t2)
t2

## Parameters as Free Variables (`g` here)

In [ ]:
t3 = Tree.fromstring(
'''
(S John
  (VP loves t_1)
)
''')

t3
m3 = calc.interpret(t3)
t3

m3
`m3(g={1:MARY.e})   # Set parameters using keywords

## Predicate Abstraction

In [ ]:
t4 = Tree.fromstring(
'''
(S Mary (CP 1
  (S John
    (VP loves t_1)
  )
))
''')

t4
calc.interpret(t4)
t4